In [1]:
%reload_ext autoreload
%autoreload 2
%reload_ext notexbook
%texify

### Use narsil2_env to run this file

In [2]:

from tqdm.notebook import tqdm

import os
import torch
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import ncolor, edt
import numpy as np
from pathlib import Path
from numba import njit, float32, int32, vectorize
np.seterr(invalid="ignore")
%matplotlib qt

import time

In [3]:
DATASET_PATH = '/mnt/sda1/SMLAT/data/sbgdata2/pooled_bg_data/phase_venus/'

In [4]:
MODEL_CHECKPOINT_PATH = '/home/pk/Documents/narsil2/saved_models/good_models/omnires_mm_old_new_320_long.ckpt'

In [5]:

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)


Device: cuda:0


In [6]:

from narsil2.segmentation.datasets import MMData, MMDataTest
from narsil2.segmentation.utils_omni import reconstruct_masks_cpu_omni, format_labels, clean_boundary
from narsil2.segmentation.utils_omni import fast_reconstruction
from narsil2.segmentation.transformations import OmniTestTransformations, unpadNumpyArray
from narsil2.segmentation.networks import Unet
from narsil2.segmentation.losses import OmniLoss
from narsil2.segmentation.lightning_modules import UTrainingModule
from scipy.ndimage.morphology import binary_dilation, binary_erosion
from skimage import io

In [7]:

test_data = MMDataTest(dataset_path=Path(DATASET_PATH), phase_fileformat="*.tiff", mothermachine_data=True,
                      transforms=OmniTestTransformations(return_tensors=True))

test_loader = DataLoader(test_data, batch_size=1, shuffle=False, drop_last=False, num_workers=4)

model = UTrainingModule.load_from_checkpoint(MODEL_CHECKPOINT_PATH)


In [8]:
test_data[0]

{'phase': tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]),
 'filename': '/mnt/sda1/SMLAT/data/sbgdata2/pooled_bg_data/phase_venus/Pos25_img_000000000.tiff',
 'raw_shape': (1041, 1302)}

In [9]:
test_data.plot(0)

In [10]:
len(test_data)

601

In [11]:
MASK_SAVE_PATH = Path('/mnt/sda1/SMLAT/data/sbgdata2/pooled_bg_data/phase_venus_mask/')
cell_prob_threshold=0.0

In [12]:
model = model.to('cuda:0')

In [13]:
#%%timeit
unpadder = unpadNumpyArray()
with torch.no_grad():
    for datapoint in test_data:
        filename = Path(datapoint['filename']).stem + ".png"
        write_path = MASK_SAVE_PATH/filename
        if not write_path.is_file():
            out = model(datapoint['phase'].unsqueeze(0).cuda())
            generated_mask = reconstruct_masks_cpu_omni(out.cpu().numpy()[0],
                                                       cell_prob_threshold=cell_prob_threshold,
                                                       min_size=100,
                                                       clean_mask=True,
                                                       fast=True)
            generated_mask = unpadder(generated_mask, datapoint['raw_shape'])
            io.imsave(write_path, generated_mask, check_contrast=False)
            print(write_path)
        else:
            print(write_path, '---exists')

Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
/mnt/sda1/SMLAT/data/sbgdata2/pooled_bg_data/phase_venus_mask/Pos25_img_000000000.png
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - fou

Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
/mnt/sda1/SMLAT/data/sbgdata2/pooled_bg_data/phase_venus_mask/Pos25_img_000000015.png
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - fou

Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
/mnt/sda1/SMLAT/data/sbgdata2/pooled_bg_data/phase_venus_mask/Pos25_img_000000029.png
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
/mnt/sda1/SML

Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
/mnt/sda1/SMLAT/data/sbgdata2/pooled_bg_data/phase_venus_mask/Pos25_img_000000044.png
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
/mnt/sda1/SMLAT/data/sbgdata2/pooled_bg_data/phase_venus_mask/Pos2

Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
/mnt/sda1/SMLAT/data/sbgdata2/pooled_bg_data/phase_venus_mask/Pos25_img_000000060.png
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - fou

...

Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removi

Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removi

Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
/mnt/sda1/SMLAT/data/sbgdata2/pooled_bg_data/phase_venus_mask/Pos25_img_000000597.png
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - fou

Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removi

Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removing it.
Warning - found mask area less than 100
Removi

In [14]:
print(123)

123
